In [1]:
import numpy as np
import os
from scipy import integrate

In [9]:
# Define the reference origin
origin = 0


# Function to read velocity data, including turbulence intensity, from CSV files
def read_velocity_data(folder, case):
    coordinates = []  # List to store coordinates
    u_mean_velocities = []  # List to store mean velocities
    u_turbulence_intensities = []  # List to store turbulence intensities
    folder_path = f"data/{folder}"

    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith("_summary.csv"):
            # Extract the coordinate from the filename (e.g., data20.00_summary.csv -> 20.00mm)
            coordinate = float(
                filename.split(f"{folder}_vertical_0deg_{case}_17Hz_xc200")[1].split(
                    "_"
                )[0]
            )
            # Adjust the coordinate relative to the origin (308.5mm)
            relative_coordinate = coordinate - origin
            coordinates.append(relative_coordinate)

            # Read the CSV file
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r") as file:
                lines = file.readlines()

                # Look for the line containing U direction mean velocity and turbulence intensity
                for line in lines:
                    if line.startswith("U"):
                        # Split the line by commas, 2nd is mean velocity, 6th is turbulence intensity
                        data = line.split(",")
                        u_mean_velocity = float(data[1])
                        u_turbulence_intensity = float(
                            data[5]
                        )  # 6th element is turbulence intensity
                        u_mean_velocities.append(u_mean_velocity)
                        u_turbulence_intensities.append(u_turbulence_intensity)
                        break

    # Sort the data based on coordinates
    sorted_data = sorted(zip(coordinates, u_mean_velocities, u_turbulence_intensities))
    coordinates, u_mean_velocities, u_turbulence_intensities = zip(*sorted_data)

    return coordinates, u_mean_velocities, u_turbulence_intensities


In [5]:
case1 = "nb"
case2 = "cc"

# List of folders to compare (up to 10 folders)
# folders1 = [
#     # "20241222_01",
#     "20241222_04",
#     "20241222_06",
#     "20241222_07",
#     "20241222_09",
#     # "20241222_12",
# ]

# folders2 = [
#     # "20241222_02",
#     # "20241222_03",
#     "20241222_05",
#     "20241222_08",
#     "20241222_10",
#     "20241222_11",
# ]

folders1 = [
    # nb
    "20250121_01",
    "20250121_04",
    "20250121_06",
    "20250121_07",
]

folders2 = [
    # cc
    "20250121_02",
    "20250121_03",
    "20250121_05",
    "20250121_08",
]

# folders2 = [
#     # wb
#     "20241217_09",
#     "20241217_12",
#     "20241217_16",
#     "20241217_21",
# ]

# folders2 = [
#     # ws
#     "20250121_01",
#     "20250121_04",
#     "20250121_06",
#     "20250121_07",
# ]

In [23]:
# Load HW data from CSV files

# Initialize lists to store data for averaging
all_mean_velocities_case1 = []
all_mean_velocities_case2 = []

# Loop through folders and collect mean velocity data for case1
for folder in folders1:
    coordinates, u_mean_velocities, _ = read_velocity_data(folder, case1)
    all_mean_velocities_case1.append(u_mean_velocities)

# Loop through folders and collect mean velocity data for case2
for folder in folders2:
    _, u_mean_velocities, _ = read_velocity_data(folder, case2)
    all_mean_velocities_case2.append(u_mean_velocities)

# Convert coordinates to meters
coordinates = np.array(coordinates) * 1e-3

# Convert lists to arrays for easier averaging
all_mean_velocities_case1 = np.array(all_mean_velocities_case1)
all_mean_velocities_case2 = np.array(all_mean_velocities_case2)

# Compute the mean, max, and min mean velocities across folders for case1
mean_velocities_case1 = np.mean(all_mean_velocities_case1, axis=0)
mean_velocities_case2 = np.mean(all_mean_velocities_case2, axis=0)


In [5]:
# Load PT data from CSV files

In [6]:
# Compare flow rate of HW and PT

Q_hw_trapz = integrate.trapezoid(mean_velocities_case1, coordinates)
Q_hw_simps = integrate.simpson(mean_velocities_case1, coordinates)

In [7]:
z_integrate_trapz = integrate.trapezoid(mean_velocities_case1**2, coordinates)
z_integrate_simps = integrate.simpson(mean_velocities_case1**2, coordinates)

print(z_integrate_trapz)
print(z_integrate_simps)

248.00462042296465
248.05590902026628
